In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from functools import reduce
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 120)
DATA_DIR = Path('data')
OUT_DIR = Path('processed')
OUT_DIR.mkdir(exist_ok=True)

In [ ]:
# Load datasets
climate_files = [
    'Huye_climate_data.csv','Kigali_climate_data.csv','Muhanga_climate_data.csv',
    'Kibungo_climate_data.csv','Musanze_climate_data.csv','Nyagatare_climate_data.csv',
    'Rubavu_climate_data.csv','Rusizi_climate_data.csv'
]

climate_dfs = {}
for f in climate_files:
    loc = f.split('_')[0].lower()
    df = pd.read_csv(DATA_DIR / f, parse_dates=['Date'], na_values=['-999.0', -999.0])
    df.replace(-999.0, np.nan, inplace=True)
    climate_dfs[loc] = df

tomato = pd.read_csv(DATA_DIR / 'Tomato_price.csv', parse_dates=['Date'])
harvest = pd.read_csv(DATA_DIR / 'harvest_dataset.csv', parse_dates=['Date'])

print(f"Loaded {len(climate_dfs)} climate datasets, tomato: {tomato.shape}, harvest: {harvest.shape}")

In [ ]:
# 1. DATA CLEANING
missing_before = sum(df.isnull().sum().sum() for df in climate_dfs.values()) + tomato.isnull().sum().sum() + harvest.isnull().sum().sum()
duplicates_before = sum(df.duplicated().sum() for df in climate_dfs.values()) + tomato.duplicated().sum() + harvest.duplicated().sum()

# Clean climate data
for loc, df in climate_dfs.items():
    df.drop_duplicates(inplace=True)
    for col in df.select_dtypes(include='number').columns:
        df[col] = df[col].fillna(df[col].median())

# Clean tomato and harvest
tomato.drop_duplicates(inplace=True)
tomato['Average'] = pd.to_numeric(tomato['Average'], errors='coerce')
tomato['Average'] = tomato['Average'].fillna(tomato['Average'].median())

harvest.drop_duplicates(inplace=True)
harvest['Date'] = pd.to_datetime(harvest['Date']).dt.tz_localize(None)
for col in harvest.select_dtypes(include='number').columns:
    harvest[col] = harvest[col].fillna(harvest[col].median())

missing_after = sum(df.isnull().sum().sum() for df in climate_dfs.values()) + tomato.isnull().sum().sum() + harvest.isnull().sum().sum()
duplicates_after = sum(df.duplicated().sum() for df in climate_dfs.values()) + tomato.duplicated().sum() + harvest.duplicated().sum()

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.bar(['Before', 'After'], [missing_before, missing_after])
plt.title('Missing Values Before vs After Cleaning')
plt.ylabel('Total Missing Values')

plt.subplot(1,2,2)
plt.bar(['Before', 'After'], [duplicates_before, duplicates_after])
plt.title('Duplicate Rows Before vs After Cleaning')
plt.ylabel('Total Duplicates')
plt.tight_layout()
plt.show()

In [ ]:
# 2. DATA INTEGRATION
# Rename and aggregate climate data
for loc, df in climate_dfs.items():
    df.rename(columns={'T2M':'temp_c','PRECTOTCORR':'precip_mm','WS2M':'wind_m_s','RH2M':'rh_pct','ALLSKY_SFC_SW_DWN':'sw_down'}, inplace=True)
    df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
    climate_dfs[loc] = df.groupby('Date').agg({'temp_c':'mean','precip_mm':'sum','wind_m_s':'mean','rh_pct':'mean','sw_down':'mean'}).reset_index()

# Merge all climate stations
all_daily = [df.rename(columns=lambda c: f"{loc}_{c}" if c!='Date' else c) for loc, df in climate_dfs.items()]
climate_all = reduce(lambda a,b: a.merge(b,on='Date',how='outer'), all_daily)

temp_cols = [c for c in climate_all.columns if c.endswith('temp_c')]
precip_cols = [c for c in climate_all.columns if c.endswith('precip_mm')]
wind_cols = [c for c in climate_all.columns if c.endswith('wind_m_s')]
rh_cols = [c for c in climate_all.columns if c.endswith('rh_pct')]

climate_avg = pd.DataFrame({
    'Date': climate_all['Date'],
    'temp_c': climate_all[temp_cols].mean(axis=1),
    'precip_mm': climate_all[precip_cols].sum(axis=1),
    'wind_m_s': climate_all[wind_cols].mean(axis=1),
    'rh_pct': climate_all[rh_cols].mean(axis=1)
}).drop_duplicates(subset=['Date']).sort_values('Date').reset_index(drop=True)

# Clean and merge tomato data
tomato.rename(columns={'Average':'price_avg'}, inplace=True)
tomato['Date'] = pd.to_datetime(tomato['Date']).dt.normalize()

# Merge harvest data
harvest_daily = harvest.groupby('Date').agg({'Harvest_kg':'sum','Price_index':'mean'}).reset_index()

# Final integration
integrated_df = tomato.merge(climate_avg, on='Date', how='left').merge(harvest_daily, on='Date', how='left')
integrated_df[['Harvest_kg','Price_index']] = integrated_df[['Harvest_kg','Price_index']].fillna(method='ffill').fillna(method='bfill')

plt.figure(figsize=(8,4))
plt.bar(['Climate', 'Tomato', 'Harvest', 'Integrated'], [len(climate_avg), len(tomato), len(harvest_daily), len(integrated_df)])
plt.title('Records in Each Dataset Before vs After Integration')
plt.ylabel('Records')
plt.show()

print(f"Integrated dataset shape: {integrated_df.shape}")

In [ ]:
# 3. DATA REDUCTION
reduce_cols = ['Date', 'price_avg', 'temp_c', 'precip_mm', 'wind_m_s', 'rh_pct', 'Harvest_kg', 'Price_index']
reduced_df = integrated_df[reduce_cols].copy()

plt.figure(figsize=(8,4))
plt.bar(['Before', 'After'], [integrated_df.shape[1], reduced_df.shape[1]])
plt.title('Number of Columns Before vs After Reduction')
plt.ylabel('Columns')
plt.show()

print(f"Reduced from {integrated_df.shape[1]} to {reduced_df.shape[1]} columns")

In [ ]:
# 4. DATA TRANSFORMATION
scaler = MinMaxScaler()
numeric_cols = ['price_avg', 'temp_c', 'precip_mm', 'wind_m_s', 'rh_pct', 'Harvest_kg', 'Price_index']
reduced_df[numeric_cols] = scaler.fit_transform(reduced_df[numeric_cols])

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.histplot(integrated_df['price_avg'].dropna(), kde=True, color='gray', label='Before')
plt.legend()
plt.title('Price Distribution Before Scaling')

plt.subplot(1,2,2)
sns.histplot(reduced_df['price_avg'], kde=True, color='blue', label='After')
plt.legend()
plt.title('Price Distribution After Scaling')
plt.tight_layout()
plt.show()

In [ ]:
# 5. DATA DISCRETIZATION
reduced_df['temp_category'] = pd.cut(reduced_df['temp_c'], bins=3, labels=['Cool', 'Moderate', 'Warm'])
reduced_df['precip_category'] = pd.cut(reduced_df['precip_mm'], bins=3, labels=['Low', 'Medium', 'High'])

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.countplot(x=reduced_df['temp_category'], palette='coolwarm')
plt.title('Temperature Categories After Discretization')
plt.ylabel('Records')

plt.subplot(1,2,2)
sns.countplot(x=reduced_df['precip_category'], palette='Blues')
plt.title('Precipitation Categories After Discretization')
plt.ylabel('Records')
plt.tight_layout()
plt.show()

In [ ]:
# 6. DATA AUGMENTATION
augmented = reduced_df.sample(frac=0.1, random_state=42)
augmented[numeric_cols] += np.random.normal(0, 0.05, size=(augmented.shape[0], len(numeric_cols)))
augmented['source'] = 'Synthetic'
reduced_df['source'] = 'Original'
augmented_df = pd.concat([reduced_df, augmented], ignore_index=True)

plt.figure(figsize=(8,4))
plt.bar(['Before', 'After'], [reduced_df.shape[0], augmented_df.shape[0]])
plt.title('Dataset Size Before vs After Augmentation')
plt.ylabel('Records')
plt.show()

print(f"Final dataset shape: {augmented_df.shape}")
print(f"Original records: {(augmented_df['source'] == 'Original').sum()}")
print(f"Synthetic records: {(augmented_df['source'] == 'Synthetic').sum()}")

In [ ]:
# Save processed data
augmented_df.to_csv(OUT_DIR / 'processed_farmer_data.csv', index=False)
print(f"Processed data saved to {OUT_DIR / 'processed_farmer_data.csv'}")
print("\nFinal dataset summary:")
print(augmented_df.info())
print("\nFirst 5 rows:")
print(augmented_df.head())